In [1]:
import scanpy as sc
import numpy as np
import pandas as pd

# Read EXP and metadata as csv

In [2]:
experiment_id = 'exp_1'
save_name = './experiment_1.h5ad'

In [3]:
# load in csv file
exp = pd.read_csv('/Users/emilyjones/Desktop/HCA/project/ex_mat/exp_1_ortholog_deleted_rows.csv',index_col = 0)
exp

,symbol,biotype,chromosome,start,stop,CCR1_R1,CCR1_R2,CCR1_R3,CCR2_R1,CCR2_R2,...,NO_CCR_mean,CCR1_median,CCR2_median,CCR5_median,NO_CCR_median,CCR1_stdev,CCR2_stdev,CCR5_stdev,NO_CCR_stdev,HGNC_orth
ID,,,,,,,,,,,,,,,,,,,,,
ENSMUSG00000028180,ZRANB2,GENE,3,157239797,157254047,986.75,1036.86,995.52,670.33,733.58,...,1324.893333,995.52,670.33,872.07,1369.03,21.850300,53.850579,69.952720,111.754387,ZRANB2
ENSMUSG00000028182,LRRIQ3,GENE,3,154799071,154899917,0.00,1.81,5.26,1.85,6.55,...,1.613333,1.81,1.85,2.80,2.33,2.181900,2.757112,1.481741,1.143163,LRRIQ3
ENSMUSG00000028185,DNASE2B,GENE,3,146286740,146321351,0.00,3.63,9.64,0.00,0.00,...,3.566667,3.63,0.00,0.00,1.26,3.975293,0.428978,1.315219,4.184809,DNASE2B
ENSMUSG00000028184,ADGRL2,GENE,3,148521219,148696191,740.06,774.02,700.20,162.51,339.50,...,1032.726667,740.06,162.51,863.66,1015.64,30.168957,86.658077,176.202476,34.333416,ADGRL2
ENSMUSG00000028187,RPF1,GENE,3,146211711,146227184,198.81,230.21,190.17,330.55,307.84,...,178.436667,198.81,330.55,243.96,178.28,17.204041,11.713167,30.702674,19.437017,RPF1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSMUSG00000029581,FSCN1,GENE,5,142946098,142958940,1291.73,1335.05,1459.98,169.89,182.30,...,1062.906667,1335.05,169.89,1627.63,1130.65,71.330366,19.600075,386.823495,107.053272,FSCN1
ENSMUSG00000042672,DCST1,GENE,3,89257526,89272560,7.29,1.81,9.64,6.46,6.55,...,5.536667,7.29,6.46,3.74,4.05,3.280613,1.360498,1.526456,5.234248,DCST1
ENSMUSG00000042670,IMMP1L,GENE,2,105734983,105795903,46.84,38.07,40.31,54.48,37.12,...,37.503333,40.31,37.12,38.42,38.92,3.720385,9.709945,15.808936,6.378424,IMMP1L


# Seperate the expression values from the .var values

In [4]:
exp_mat = exp[exp.columns[~exp.columns.str.contains('stdev|median|mean|symbol|biotype|chromosome|start|stop|HGNC_orth')]] # columns to be ignored
var_pd = exp[exp.columns[~exp.columns.isin(exp_mat.columns)]]
meta = pd.DataFrame(index=exp_mat.columns)
meta['experiment_id'] =  experiment_id
meta = meta.rename_axis("index")

In [6]:
meta

,experiment_id
index,
CCR1_R1,exp_1
CCR1_R2,exp_1
CCR1_R3,exp_1
CCR2_R1,exp_1
CCR2_R2,exp_1
CCR2_R3,exp_1
CCR5_R1,exp_1
CCR5_R2,exp_1
CCR5_R3,exp_1


In [7]:
# Create data
adata = sc.AnnData(X = np.array((exp_mat).T))
# Format the vairables
adata.var = var_pd
# Push the human genes into the index of .var
adata.var['mouse_ens_id'] = adata.var.index[:]
adata.var= adata.var.rename_axis("index")
# Insert the variables, of which we should only have batch information
adata.obs = meta
adata.write(save_name)

<ipython-input-7-6fc8178e0df5>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adata.var['mouse_ens_id'] = adata.var.index[:]
/Applications/anaconda3/envs/python_env/lib/python3.9/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'experiment_id' as categorical
/Applications/anaconda3/envs/python_env/lib/python3.9/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a 

In [8]:
adata

AnnData object with n_obs × n_vars = 12 × 13665
    obs: 'experiment_id'
    var: 'symbol', 'biotype', 'chromosome', 'start', 'stop', 'CCR1_mean', 'CCR2_mean', 'CCR5_mean', 'NO_CCR_mean', 'CCR1_median', 'CCR2_median', 'CCR5_median', 'NO_CCR_median', 'CCR1_stdev', 'CCR2_stdev', 'CCR5_stdev', 'NO_CCR_stdev', 'HGNC_orth', 'mouse_ens_id'

In [9]:
adata.obs

,experiment_id
index,
CCR1_R1,exp_1
CCR1_R2,exp_1
CCR1_R3,exp_1
CCR2_R1,exp_1
CCR2_R2,exp_1
CCR2_R3,exp_1
CCR5_R1,exp_1
CCR5_R2,exp_1
CCR5_R3,exp_1


In [10]:
adata.var

,symbol,biotype,chromosome,start,stop,CCR1_mean,CCR2_mean,CCR5_mean,NO_CCR_mean,CCR1_median,CCR2_median,CCR5_median,NO_CCR_median,CCR1_stdev,CCR2_stdev,CCR5_stdev,NO_CCR_stdev,HGNC_orth,mouse_ens_id
index,,,,,,,,,,,,,,,,,,,
ENSMUSG00000028180,ZRANB2,GENE,3,157239797,157254047,1006.376667,668.540000,836.756667,1324.893333,995.52,670.33,872.07,1369.03,21.850300,53.850579,69.952720,111.754387,ZRANB2,ENSMUSG00000028180
ENSMUSG00000028182,LRRIQ3,GENE,3,154799071,154899917,2.356667,2.800000,3.066667,1.613333,1.81,1.85,2.80,2.33,2.181900,2.757112,1.481741,1.143163,LRRIQ3,ENSMUSG00000028182
ENSMUSG00000028185,DNASE2B,GENE,3,146286740,146321351,4.423333,0.303333,0.930000,3.566667,3.63,0.00,0.00,1.26,3.975293,0.428978,1.315219,4.184809,DNASE2B,ENSMUSG00000028185
ENSMUSG00000028184,ADGRL2,GENE,3,148521219,148696191,738.093333,217.176667,878.693333,1032.726667,740.06,162.51,863.66,1015.64,30.168957,86.658077,176.202476,34.333416,ADGRL2,ENSMUSG00000028184
ENSMUSG00000028187,RPF1,GENE,3,146211711,146227184,206.396667,324.256667,226.490000,178.436667,198.81,330.55,243.96,178.28,17.204041,11.713167,30.702674,19.437017,RPF1,ENSMUSG00000028187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSMUSG00000029581,FSCN1,GENE,5,142946098,142958940,1362.253333,162.706667,1879.953333,1062.906667,1335.05,169.89,1627.63,1130.65,71.330366,19.600075,386.823495,107.053272,FSCN1,ENSMUSG00000029581
ENSMUSG00000042672,DCST1,GENE,3,89257526,89272560,6.246667,5.543333,3.890000,5.536667,7.29,6.46,3.74,4.05,3.280613,1.360498,1.526456,5.234248,DCST1,ENSMUSG00000042672
ENSMUSG00000042670,IMMP1L,GENE,2,105734983,105795903,41.740000,41.106667,44.650000,37.503333,40.31,37.12,38.42,38.92,3.720385,9.709945,15.808936,6.378424,IMMP1L,ENSMUSG00000042670


In [11]:
adata.X

array([[9.86750e+02, 0.00000e+00, 0.00000e+00, ..., 4.68400e+01,
        5.68840e+03, 6.40140e+02],
       [1.03686e+03, 1.81000e+00, 3.63000e+00, ..., 3.80700e+01,
        4.99488e+03, 5.69190e+02],
       [9.95520e+02, 5.26000e+00, 9.64000e+00, ..., 4.03100e+01,
        6.43847e+03, 6.48490e+02],
       ...,
       [1.43425e+03, 2.33000e+00, 0.00000e+00, ..., 2.90800e+01,
        4.87041e+03, 7.17710e+02],
       [1.36903e+03, 0.00000e+00, 9.44000e+00, ..., 4.45100e+01,
        3.72806e+03, 5.61100e+02],
       [1.17140e+03, 2.51000e+00, 1.26000e+00, ..., 3.89200e+01,
        4.96685e+03, 6.45340e+02]], dtype=float32)

In [12]:
adata.X.shape

(12, 13665)